# CY₃ Unified Spectral Validation

**Conjecture Universelle**:
$$\lambda_1 \times H^* = \dim(\text{Hol}) - h$$

**Définition H* (Betti, comme G₂)**:
$$H^* = b_2 + b_3 + 1$$

Pour CY₃:
- $b_2 = h^{1,1}$
- $b_3 = 2(h^{2,1} + 1) = 2h^{2,1} + 2$
- Donc: $H^* = h^{1,1} + 2h^{2,1} + 3$

**Target SU(3)**: $\dim(SU(3)) - h = 8 - 2 = 6$

---

| Manifold | h¹¹ | h²¹ | b₂ | b₃ | H* | λ₁ target |
|----------|-----|-----|----|----|-----|----------|
| K₇ (G₂) | - | - | 21 | 77 | 99 | 0.131 |
| T⁶/ℤ₃ | 9 | 0 | 9 | 2 | **12** | **0.500** |
| Quintic | 1 | 101 | 1 | 204 | **206** | **0.029** |

---

In [ ]:
# Cell 1: Setup
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
import json
from datetime import datetime
import time
import matplotlib.pyplot as plt

# GPU check
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("✓ CuPy available - GPU acceleration enabled")
except ImportError:
    GPU_AVAILABLE = False
    print("✗ CuPy not available - using CPU")

print(f"NumPy version: {np.__version__}")

# JSON serializer fix
def json_serializer(obj):
    """Handle numpy types for JSON serialization."""
    if isinstance(obj, (np.bool_, np.generic)):
        return obj.item()
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

print("✓ JSON serializer ready")

In [ ]:
# Cell 2: H* Betti Definition (CORRECTED)

def compute_Hstar_Betti(h11: int, h21: int) -> dict:
    """
    Compute H* using Betti numbers (same formula as G₂).
    
    For CY₃:
        b₂ = h¹¹
        b₃ = 2(h²¹ + 1) = 2h²¹ + 2
        H* = b₂ + b₃ + 1
    """
    b2 = h11
    b3 = 2 * (h21 + 1)
    Hstar = b2 + b3 + 1
    
    return {
        "h11": h11,
        "h21": h21,
        "b2": b2,
        "b3": b3,
        "Hstar": Hstar,
        "lambda1_target": 6.0 / Hstar,
    }

# Define manifolds
MANIFOLDS = {
    "T6_Z3": {
        "name": "T⁶/ℤ₃ Orbifold",
        "h11": 9,
        "h21": 0,
        "chi": 18,
        "holonomy": "SU(3)",
        "metric": "flat orbifold",
    },
    "Quintic": {
        "name": "Quintic P⁴[5]",
        "h11": 1,
        "h21": 101,
        "chi": -200,
        "holonomy": "SU(3)",
        "metric": "Fubini-Study (ambient)",
    },
}

# Add H* data
for key, data in MANIFOLDS.items():
    hstar_data = compute_Hstar_Betti(data["h11"], data["h21"])
    data.update(hstar_data)

# Reference: G₂
G2_REFERENCE = {
    "name": "K₇ (G₂)",
    "b2": 21,
    "b3": 77,
    "Hstar": 99,
    "target": 13,
    "measured": 13.0,
    "status": "EXACT",
}

# Conjecture
TARGET_SU3 = 6  # dim(SU(3)) - h = 8 - 2
TARGET_G2 = 13  # dim(G₂) - h = 14 - 1

print("=" * 60)
print("  H* BETTI DEFINITION (CORRECTED)")
print("=" * 60)
print(f"\nFormula: H* = b₂ + b₃ + 1")
print(f"Target SU(3): λ₁ × H* = {TARGET_SU3}")
print()
print("| Manifold | h¹¹ | h²¹ | b₂ | b₃ | H* | λ₁ target |")
print("|----------|-----|-----|----|----|-----|----------|")
print(f"| K₇ (G₂)  | -   | -   | 21 | 77 | 99  | 0.1313   |")
for key, data in MANIFOLDS.items():
    print(f"| {data['name'][:10]:<10} | {data['h11']:3} | {data['h21']:3} | {data['b2']:2} | {data['b3']:3} | {data['Hstar']:3} | {data['lambda1_target']:.4f}   |")

In [ ]:
# Cell 3: T⁶/ℤ₃ Sampling

OMEGA_ANGLE = 2 * np.pi / 3

def sample_T6_Z3(N: int, seed: int = 42) -> np.ndarray:
    """Sample N points on T⁶/ℤ₃ orbifold."""
    rng = np.random.default_rng(seed)
    
    # Sample on T⁶
    angles = rng.uniform(0, 2 * np.pi, size=(N, 6)).astype(np.float32)
    
    # Project to fundamental domain (θ₁ ∈ [0, 2π/3))
    k = np.floor(angles[:, 0] / OMEGA_ANGLE).astype(int)
    for i in range(6):
        angles[:, i] = np.mod(angles[:, i] - k * OMEGA_ANGLE, 2 * np.pi)
    
    return angles


def distance_T6_Z3(angles: np.ndarray) -> np.ndarray:
    """Orbifold distance matrix on T⁶/ℤ₃."""
    N = angles.shape[0]
    D_min = np.full((N, N), np.inf, dtype=np.float32)
    
    for k in range(3):
        angles_shifted = np.mod(angles + k * OMEGA_ANGLE, 2 * np.pi)
        diff = np.abs(angles[:, None, :] - angles_shifted[None, :, :])
        diff_toric = np.minimum(diff, 2 * np.pi - diff)
        D_k = np.sqrt(np.sum(diff_toric**2, axis=2))
        D_min = np.minimum(D_min, D_k)
    
    return D_min

print("✓ T⁶/ℤ₃ sampling functions ready")

In [ ]:
# Cell 4: Quintic Sampling

def quintic_poly(z: np.ndarray) -> np.ndarray:
    """Fermat quintic: z₀⁵ + z₁⁵ + z₂⁵ + z₃⁵ + z₄⁵ = 0"""
    return np.sum(z**5, axis=1)


def project_to_quintic(z: np.ndarray, max_iter: int = 20) -> np.ndarray:
    """Newton projection onto quintic."""
    for _ in range(max_iter):
        p = quintic_poly(z)
        if np.all(np.abs(p) < 1e-10):
            break
        grad = 5 * z**4
        grad_norm_sq = np.maximum(np.sum(np.abs(grad)**2, axis=1, keepdims=True), 1e-20)
        z = z - (p[:, None] * np.conj(grad)) / grad_norm_sq
        z = z / np.linalg.norm(z, axis=1, keepdims=True)
    return z


def sample_quintic(N: int, seed: int = 42) -> np.ndarray:
    """Sample N points on Fermat quintic."""
    rng = np.random.default_rng(seed)
    
    points = []
    batch_size = min(N * 2, 10000)
    
    while len(points) < N:
        z = rng.standard_normal((batch_size, 5)) + 1j * rng.standard_normal((batch_size, 5))
        z = z / np.linalg.norm(z, axis=1, keepdims=True)
        z_proj = project_to_quintic(z)
        valid = np.abs(quintic_poly(z_proj)) < 1e-8
        points.extend(z_proj[valid])
    
    return np.array(points[:N])


def distance_quintic(z: np.ndarray) -> np.ndarray:
    """Fubini-Study distance on CP⁴."""
    inner = z @ z.conj().T
    cos_dist = np.clip(np.abs(inner), 0, 1)
    return np.arccos(cos_dist).astype(np.float32)

print("✓ Quintic sampling functions ready")

In [ ]:
# Cell 5: Graph Laplacian & Eigenvalue

def compute_lambda1(D: np.ndarray, k: int = 25) -> float:
    """
    Build graph Laplacian and compute λ₁.
    """
    N = D.shape[0]
    k = min(k, N - 1)
    
    # Sigma from k-NN distances
    knn_dists = np.partition(D, k, axis=1)[:, 1:k+1]
    sigma = max(np.median(knn_dists), 1e-10)
    
    # Gaussian weights
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    
    # Sparsify to k-NN
    for i in range(N):
        idx_keep = np.argpartition(-W[i], k)[:k]
        mask = np.ones(N, dtype=bool)
        mask[idx_keep] = False
        W[i, mask] = 0
    
    W = (W + W.T) / 2
    
    # Symmetric normalized Laplacian
    d = np.maximum(W.sum(axis=1), 1e-10)
    d_inv_sqrt = 1.0 / np.sqrt(d)
    L = np.eye(N) - np.outer(d_inv_sqrt, d_inv_sqrt) * W
    L_sparse = sp.csr_matrix(L)
    
    # Compute eigenvalues
    try:
        eigs, _ = eigsh(L_sparse, k=5, which='SM', tol=1e-8)
        eigs = np.sort(np.real(eigs))
        for ev in eigs:
            if ev > 1e-8:
                return float(ev)
        return float(eigs[1]) if len(eigs) > 1 else 0.0
    except:
        return np.nan

print("✓ Laplacian functions ready")

In [ ]:
# Cell 6: Validation Parameters

# Grid parameters
N_VALUES = [2000, 5000, 10000, 20000]
K_VALUES = [40, 60, 100, 150, 200]
SEEDS = [42, 123, 456]

# Tolerance
TOLERANCE = 0.20  # 20%

print("Validation Grid:")
print(f"  N: {N_VALUES}")
print(f"  k: {K_VALUES}")
print(f"  Seeds: {SEEDS}")
print(f"  Total tests per manifold: {len(N_VALUES) * len(K_VALUES) * len(SEEDS)}")

In [ ]:
# Cell 7: Run T⁶/ℤ₃ Validation
print("=" * 60)
print("  T⁶/ℤ₃ ORBIFOLD VALIDATION")
print("=" * 60)

data_T6Z3 = MANIFOLDS["T6_Z3"]
Hstar_T6Z3 = data_T6Z3["Hstar"]
print(f"H* (Betti) = {Hstar_T6Z3}")
print(f"Target: λ₁ × {Hstar_T6Z3} = {TARGET_SU3}")
print(f"λ₁ target = {data_T6Z3['lambda1_target']:.4f}")
print()

results_T6Z3 = []
t_start = time.time()

for N in N_VALUES:
    print(f"N = {N}...")
    for seed in SEEDS:
        # Sample once
        angles = sample_T6_Z3(N, seed=seed)
        D = distance_T6_Z3(angles)
        
        for k in K_VALUES:
            t0 = time.time()
            lambda1 = compute_lambda1(D, k=k)
            product = lambda1 * Hstar_T6Z3
            deviation = abs(product - TARGET_SU3) / TARGET_SU3 * 100
            
            results_T6Z3.append({
                "manifold": "T6_Z3",
                "N": N,
                "k": k,
                "seed": seed,
                "Hstar": Hstar_T6Z3,
                "lambda1": float(lambda1),
                "product": float(product),
                "target": TARGET_SU3,
                "deviation_pct": float(deviation),
                "passed": bool(deviation < TOLERANCE * 100),
                "time_s": time.time() - t0,
            })
        
        # Summary for this (N, seed)
        best_k = max(K_VALUES)
        r = [x for x in results_T6Z3 if x["N"]==N and x["seed"]==seed and x["k"]==best_k][0]
        print(f"  seed={seed}, k={best_k}: λ₁={r['lambda1']:.4f}, product={r['product']:.2f}")

print(f"\nTotal time: {(time.time() - t_start)/60:.1f} min")

In [ ]:
# Cell 8: Run Quintic Validation
print("=" * 60)
print("  QUINTIC VALIDATION")
print("=" * 60)

data_Quintic = MANIFOLDS["Quintic"]
Hstar_Quintic = data_Quintic["Hstar"]
print(f"H* (Betti) = {Hstar_Quintic}")
print(f"Target: λ₁ × {Hstar_Quintic} = {TARGET_SU3}")
print(f"λ₁ target = {data_Quintic['lambda1_target']:.5f}")
print()

results_Quintic = []
t_start = time.time()

for N in N_VALUES:
    print(f"N = {N}...")
    for seed in SEEDS:
        # Sample once
        z = sample_quintic(N, seed=seed)
        D = distance_quintic(z)
        
        for k in K_VALUES:
            t0 = time.time()
            lambda1 = compute_lambda1(D, k=k)
            product = lambda1 * Hstar_Quintic
            deviation = abs(product - TARGET_SU3) / TARGET_SU3 * 100
            
            results_Quintic.append({
                "manifold": "Quintic",
                "N": N,
                "k": k,
                "seed": seed,
                "Hstar": Hstar_Quintic,
                "lambda1": float(lambda1),
                "product": float(product),
                "target": TARGET_SU3,
                "deviation_pct": float(deviation),
                "passed": bool(deviation < TOLERANCE * 100),
                "time_s": time.time() - t0,
            })
        
        # Summary
        best_k = max(K_VALUES)
        r = [x for x in results_Quintic if x["N"]==N and x["seed"]==seed and x["k"]==best_k][0]
        print(f"  seed={seed}, k={best_k}: λ₁={r['lambda1']:.5f}, product={r['product']:.2f}")

print(f"\nTotal time: {(time.time() - t_start)/60:.1f} min")

In [ ]:
# Cell 9: Combined Analysis
print("=" * 60)
print("  COMBINED ANALYSIS")
print("=" * 60)

def analyze_results(results, name):
    """Analyze results for a manifold."""
    products = [r["product"] for r in results]
    high_N = [r for r in results if r["N"] >= 10000]
    high_N_products = [r["product"] for r in high_N] if high_N else products
    
    # Best result (highest k, highest N)
    best = max(results, key=lambda r: (r["N"], r["k"]))
    
    return {
        "name": name,
        "Hstar": results[0]["Hstar"],
        "mean_product": float(np.mean(products)),
        "std_product": float(np.std(products)),
        "high_N_mean": float(np.mean(high_N_products)),
        "high_N_std": float(np.std(high_N_products)),
        "best_product": float(best["product"]),
        "best_N": best["N"],
        "best_k": best["k"],
        "deviation_pct": float(abs(np.mean(high_N_products) - TARGET_SU3) / TARGET_SU3 * 100),
    }

analysis_T6Z3 = analyze_results(results_T6Z3, "T⁶/ℤ₃")
analysis_Quintic = analyze_results(results_Quintic, "Quintic")

print(f"\nTarget: λ₁ × H* = {TARGET_SU3} (SU(3) holonomy)")
print()
print("| Manifold | H* | High-N Mean | Std | Best | Deviation |")
print("|----------|-----|-------------|-----|------|-----------|")
for a in [analysis_T6Z3, analysis_Quintic]:
    print(f"| {a['name']:<8} | {a['Hstar']:3} | {a['high_N_mean']:11.2f} | {a['high_N_std']:.2f} | {a['best_product']:.2f} | {a['deviation_pct']:.1f}% |")

print("\n" + "-" * 40)
print("Reference: K₇ (G₂)")
print(f"  H* = 99, λ₁ × H* = 13.0 (EXACT)")
print("-" * 40)

In [ ]:
# Cell 10: Visualization
print("Generating plots...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: T⁶/ℤ₃ - Product vs k for each N
ax1 = axes[0, 0]
for N in N_VALUES:
    subset = [r for r in results_T6Z3 if r["N"] == N]
    by_k = {}
    for r in subset:
        by_k.setdefault(r["k"], []).append(r["product"])
    ks = sorted(by_k.keys())
    means = [np.mean(by_k[k]) for k in ks]
    ax1.plot(ks, means, 'o-', label=f'N={N}')

ax1.axhline(y=TARGET_SU3, color='r', linestyle='--', linewidth=2, label=f'Target = {TARGET_SU3}')
ax1.axhspan(TARGET_SU3 * 0.8, TARGET_SU3 * 1.2, alpha=0.2, color='green')
ax1.set_xlabel('k (neighbors)')
ax1.set_ylabel('λ₁ × H*')
ax1.set_title(f'T⁶/ℤ₃: H* = {analysis_T6Z3["Hstar"]} (Betti)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Quintic - Product vs k for each N
ax2 = axes[0, 1]
for N in N_VALUES:
    subset = [r for r in results_Quintic if r["N"] == N]
    by_k = {}
    for r in subset:
        by_k.setdefault(r["k"], []).append(r["product"])
    ks = sorted(by_k.keys())
    means = [np.mean(by_k[k]) for k in ks]
    ax2.plot(ks, means, 'o-', label=f'N={N}')

ax2.axhline(y=TARGET_SU3, color='r', linestyle='--', linewidth=2, label=f'Target = {TARGET_SU3}')
ax2.axhspan(TARGET_SU3 * 0.8, TARGET_SU3 * 1.2, alpha=0.2, color='green')
ax2.set_xlabel('k (neighbors)')
ax2.set_ylabel('λ₁ × H*')
ax2.set_title(f'Quintic: H* = {analysis_Quintic["Hstar"]} (Betti)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Both manifolds - Product vs N (best k)
ax3 = axes[1, 0]
best_k = max(K_VALUES)

for results, name, color in [(results_T6Z3, 'T⁶/ℤ₃', 'blue'), (results_Quintic, 'Quintic', 'orange')]:
    subset = [r for r in results if r["k"] == best_k]
    by_N = {}
    for r in subset:
        by_N.setdefault(r["N"], []).append(r["product"])
    Ns = sorted(by_N.keys())
    means = [np.mean(by_N[N]) for N in Ns]
    stds = [np.std(by_N[N]) for N in Ns]
    ax3.errorbar(Ns, means, yerr=stds, fmt='o-', color=color, label=name, capsize=3)

ax3.axhline(y=TARGET_SU3, color='r', linestyle='--', linewidth=2, label=f'Target = {TARGET_SU3}')
ax3.axhline(y=TARGET_G2, color='purple', linestyle=':', linewidth=2, label=f'G₂ target = {TARGET_G2}')
ax3.set_xlabel('N')
ax3.set_ylabel('λ₁ × H*')
ax3.set_title(f'Convergence (k={best_k})')
ax3.set_xscale('log')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Universal Law Comparison
ax4 = axes[1, 1]
manifolds_compare = [
    {"name": "K₇ (G₂)", "Hstar": 99, "product": 13.0, "target": 13, "color": "green"},
    {"name": "T⁶/ℤ₃", "Hstar": analysis_T6Z3["Hstar"], "product": analysis_T6Z3["high_N_mean"], "target": 6, "color": "blue"},
    {"name": "Quintic", "Hstar": analysis_Quintic["Hstar"], "product": analysis_Quintic["high_N_mean"], "target": 6, "color": "orange"},
]

x_pos = range(len(manifolds_compare))
bars = ax4.bar(x_pos, [m["product"] for m in manifolds_compare], 
               color=[m["color"] for m in manifolds_compare], alpha=0.7, label='Measured')

# Add target markers
for i, m in enumerate(manifolds_compare):
    ax4.hlines(y=m["target"], xmin=i-0.4, xmax=i+0.4, colors='red', linestyles='--', linewidth=2)

ax4.set_xticks(x_pos)
ax4.set_xticklabels([f"{m['name']}\nH*={m['Hstar']}" for m in manifolds_compare])
ax4.set_ylabel('λ₁ × H*')
ax4.set_title('Universal Law: λ₁ × H* = dim(Hol) - h')
ax4.legend(['Target', 'Measured'])
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('CY3_unified_validation.png', dpi=150)
plt.show()

print("✓ Saved: CY3_unified_validation.png")

In [ ]:
# Cell 11: Final Summary
print("=" * 60)
print("  FINAL SUMMARY: UNIVERSAL SPECTRAL LAW")
print("=" * 60)

print("\nConjecture: λ₁ × H* = dim(Hol) - h")
print("H* = b₂ + b₃ + 1 (Betti numbers)")
print()

print("| Manifold | Hol | dim | h | H* | Target | Measured | Status |")
print("|----------|-----|-----|---|-----|--------|----------|--------|")
print(f"| K₇       | G₂  | 14  | 1 | 99  | 13     | 13.0     | EXACT ✓ |")
print(f"| T⁶/ℤ₃    | SU(3)| 8  | 2 | {analysis_T6Z3['Hstar']:3} | 6      | {analysis_T6Z3['high_N_mean']:.1f}      | {'CLOSE' if analysis_T6Z3['deviation_pct'] < 50 else 'FAR'} |")
print(f"| Quintic  | SU(3)| 8  | 2 | {analysis_Quintic['Hstar']:3} | 6      | {analysis_Quintic['high_N_mean']:.1f}     | {'CLOSE' if analysis_Quintic['deviation_pct'] < 50 else 'FAR'} |")

print("\n" + "-" * 60)
print("INTERPRETATION:")
print("-" * 60)

T6Z3_ratio = analysis_T6Z3['high_N_mean'] / TARGET_SU3
Quintic_ratio = analysis_Quintic['high_N_mean'] / TARGET_SU3

print(f"\nT⁶/ℤ₃:  measured/target = {T6Z3_ratio:.2f}x")
print(f"Quintic: measured/target = {Quintic_ratio:.2f}x")

if T6Z3_ratio < 1.5 and Quintic_ratio < 1.5:
    print("\n→ Both CY₃ are within 50% of target. Promising!")
elif T6Z3_ratio < 2 or Quintic_ratio < 2:
    print("\n→ Results are in the right ballpark but not precise.")
    print("  Possible issues: metric quality, convergence, or formula adjustment needed.")
else:
    print("\n→ Significant deviation. The simple formula may need modification for CY₃.")
    print("  Or: the graph Laplacian method needs adaptation.")

In [ ]:
# Cell 12: Save Results
output = {
    "metadata": {
        "title": "CY₃ Unified Spectral Validation",
        "timestamp": datetime.now().isoformat(),
        "conjecture": "λ₁ × H* = dim(Hol) - h",
        "Hstar_formula": "b₂ + b₃ + 1 (Betti)",
        "target_SU3": TARGET_SU3,
        "target_G2": TARGET_G2,
        "N_values": N_VALUES,
        "k_values": K_VALUES,
        "seeds": SEEDS,
        "tolerance": TOLERANCE,
    },
    "manifolds": {
        "G2_reference": G2_REFERENCE,
        "T6_Z3": MANIFOLDS["T6_Z3"],
        "Quintic": MANIFOLDS["Quintic"],
    },
    "results": {
        "T6_Z3": results_T6Z3,
        "Quintic": results_Quintic,
    },
    "analysis": {
        "T6_Z3": analysis_T6Z3,
        "Quintic": analysis_Quintic,
    },
    "summary": {
        "G2_status": "EXACT (13.0)",
        "T6_Z3_status": f"measured={analysis_T6Z3['high_N_mean']:.2f}, deviation={analysis_T6Z3['deviation_pct']:.1f}%",
        "Quintic_status": f"measured={analysis_Quintic['high_N_mean']:.2f}, deviation={analysis_Quintic['deviation_pct']:.1f}%",
    },
}

filename = "CY3_unified_validation_results.json"
with open(filename, "w") as f:
    json.dump(output, f, indent=2, default=json_serializer)

print(f"\n✓ Saved: {filename}")
print("\nDownload this file + the plot and share with Claude!")